# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)


## Generate Cities List

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
# output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [3]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

661

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [4]:
# Make the API call try to limit with rate limiter or Max function
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "Imperial"

# Build query URL, rate limiter not working
query_url = f"{url}appid={weather_api_key}&units={units}&q="
# rate_limiter = RateLimiter(max_calls=50, period=60)


response = requests.get(query_url + "New York")
city_data = response.json()
city_data


{'coord': {'lon': -74.01, 'lat': 40.71},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01d'}],
 'base': 'stations',
 'main': {'temp': 76.5,
  'feels_like': 75.52,
  'temp_min': 75,
  'temp_max': 78.01,
  'pressure': 1012,
  'humidity': 44},
 'visibility': 10000,
 'wind': {'speed': 3.36, 'deg': 0},
 'clouds': {'all': 1},
 'dt': 1600881265,
 'sys': {'type': 1,
  'id': 4610,
  'country': 'US',
  'sunrise': 1600857899,
  'sunset': 1600901489},
 'timezone': -14400,
 'id': 5128581,
 'name': 'New York',
 'cod': 200}

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [5]:
# Starting URL for Weather Map API Call
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

# List of city data
city_data = []

# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in our list
for i, city in enumerate(cities):
        
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 0

    # Create endpoint URL with each city
    city_url = url + "&q=" + city
    
    # Log the url, record, and set numbers
    print("Processing Record %s of Set %s | %s" % (record_count, set_count, city))

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()

        # Parse out the max temp, humidity, and cloudiness
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_date = city_weather["dt"]

        # Append the City information into city_data list
        city_data.append({"City": city, 
                          "Lat": city_lat, 
                          "Lng": city_lng, 
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass
              
# Indicate that Data Loading is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | kaeo
Processing Record 2 of Set 1 | tautira
Processing Record 3 of Set 1 | kampot
Processing Record 4 of Set 1 | klaksvik
Processing Record 5 of Set 1 | benguela
Processing Record 6 of Set 1 | barrow
Processing Record 7 of Set 1 | punta arenas
Processing Record 8 of Set 1 | juegang
Processing Record 9 of Set 1 | hermanus
Processing Record 10 of Set 1 | castro
Processing Record 11 of Set 1 | busselton
Processing Record 12 of Set 1 | nizhneyansk
City not found. Skipping...
Processing Record 13 of Set 1 | kilcormac
Processing Record 14 of Set 1 | terny
Processing Record 15 of Set 1 | hilo
Processing Record 16 of Set 1 | yellowknife
Processing Record 17 of Set 1 | mataura
Processing Record 18 of Set 1 | portland
Processing Record 19 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 20 of Set 1 | mahebourg
Processing Record 21 of Set 1 | jaque
Processing Record 22 of Set 1 | butaritar

Processing Record 36 of Set 4 | grand river south east
City not found. Skipping...
Processing Record 37 of Set 4 | ozgon
City not found. Skipping...
Processing Record 38 of Set 4 | irbil
Processing Record 39 of Set 4 | oistins
Processing Record 40 of Set 4 | abonnema
Processing Record 41 of Set 4 | koutiala
Processing Record 42 of Set 4 | severo-kurilsk
Processing Record 43 of Set 4 | san patricio
Processing Record 44 of Set 4 | lalmohan
Processing Record 45 of Set 4 | luanda
Processing Record 46 of Set 4 | kampong chhnang
Processing Record 47 of Set 4 | la rochelle
Processing Record 48 of Set 4 | pemangkat
Processing Record 49 of Set 4 | salalah
Processing Record 0 of Set 5 | nelson bay
Processing Record 1 of Set 5 | azanka
Processing Record 2 of Set 5 | mahibadhoo
Processing Record 3 of Set 5 | dumas
Processing Record 4 of Set 5 | hami
Processing Record 5 of Set 5 | guiyang
Processing Record 6 of Set 5 | esperance
Processing Record 7 of Set 5 | bonfim
Processing Record 8 of Set 5 | w

Processing Record 29 of Set 8 | bedesa
Processing Record 30 of Set 8 | tome-acu
City not found. Skipping...
Processing Record 31 of Set 8 | dubbo
Processing Record 32 of Set 8 | mayumba
Processing Record 33 of Set 8 | sur
Processing Record 34 of Set 8 | waipawa
Processing Record 35 of Set 8 | grand-lahou
Processing Record 36 of Set 8 | lengshuitan
Processing Record 37 of Set 8 | nome
Processing Record 38 of Set 8 | half moon bay
Processing Record 39 of Set 8 | kulhudhuffushi
Processing Record 40 of Set 8 | tarata
Processing Record 41 of Set 8 | buckingham
Processing Record 42 of Set 8 | beloha
Processing Record 43 of Set 8 | mandalgovi
Processing Record 44 of Set 8 | mosquera
Processing Record 45 of Set 8 | paramonga
Processing Record 46 of Set 8 | hof
Processing Record 47 of Set 8 | trairi
Processing Record 48 of Set 8 | krasnoselkup
Processing Record 49 of Set 8 | balimo
City not found. Skipping...
Processing Record 0 of Set 9 | noumea
Processing Record 1 of Set 9 | lorengau
Processi

Processing Record 16 of Set 12 | pangody
Processing Record 17 of Set 12 | pecos
Processing Record 18 of Set 12 | waingapu
Processing Record 19 of Set 12 | ulaangom
Processing Record 20 of Set 12 | bathurst
Processing Record 21 of Set 12 | sambava
Processing Record 22 of Set 12 | kushiro
Processing Record 23 of Set 12 | opuwo
Processing Record 24 of Set 12 | sao miguel do araguaia
Processing Record 25 of Set 12 | poya
Processing Record 26 of Set 12 | sumbawa
City not found. Skipping...
Processing Record 27 of Set 12 | turan
Processing Record 28 of Set 12 | mullaitivu
City not found. Skipping...
Processing Record 29 of Set 12 | torrington
Processing Record 30 of Set 12 | saryg-sep
Processing Record 31 of Set 12 | biltine
Processing Record 32 of Set 12 | clarence town
Processing Record 33 of Set 12 | nouadhibou
Processing Record 34 of Set 12 | san cristobal
Processing Record 35 of Set 12 | sumbe
Processing Record 36 of Set 12 | pacific grove
Processing Record 37 of Set 12 | micheweni
Proc

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [6]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,kaeo,-35.10,173.78,61.0,86,45,5.01,NZ,1600881495
1,tautira,-17.73,-149.15,77.0,73,20,6.93,PF,1600881495
2,kampot,10.62,104.18,77.0,88,40,2.24,KH,1600881495
3,klaksvik,62.23,-6.59,44.6,93,59,8.05,FO,1600881495
4,benguela,-12.58,13.41,73.4,78,27,6.93,AO,1600881496


In [7]:
#  Get the indices of cities that have humidity over 100%.

humidity_max = city_data_df["Humidity"].max()
humidity_max

100

In [8]:
#  Get the indices of cities that have humidity over 100%.
city_data_df[city_data_df.Humidity > 100]

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


In [9]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".

clean_city_data = city_data_df[city_data_df.Humidity < 100]
clean_city_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,kaeo,-35.10,173.78,61.0,86,45,5.01,NZ,1600881495
1,tautira,-17.73,-149.15,77.0,73,20,6.93,PF,1600881495
2,kampot,10.62,104.18,77.0,88,40,2.24,KH,1600881495
3,klaksvik,62.23,-6.59,44.6,93,59,8.05,FO,1600881495
4,benguela,-12.58,13.41,73.4,78,27,6.93,AO,1600881496
...,...,...,...,...,...,...,...,...,...
605,usoke,-5.17,32.35,78.8,34,20,9.17,TZ,1600881613
606,don sak,9.32,99.69,81.0,86,94,2.24,TH,1600881613
607,severnyy,50.68,36.55,68.0,32,0,2.24,RU,1600881613
608,swift current,50.28,-107.80,60.8,63,1,11.41,CA,1600881613


In [10]:
# Max humidity columns greate than 100 have been removed

humidity_max = clean_city_data["Humidity"].max()
humidity_max

99

In [11]:
# Export csv file
clean_city_data.to_csv('clean_city_data.csv')
# clean_city_data.to_csv('clean_city_data.csv', index=False)

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression